In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin

In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    col_names = ['member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = 'less_bikeshare_lat_lon.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine.head()

,time_diff,temperature,humidity,windspeed,miles,temp,hum,wind,member_type_Registered,holiday_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
403526,25.0,0.640833,0.455417,0.146763,2.032180,22.119151,45.5417,9.833121,1,0,...,0,0,1,0,0,0,0,1,0,0
713156,9.0,0.348333,0.734583,0.092054,0.691932,8.371651,73.4583,6.167631,1,0,...,1,0,0,0,0,0,0,1,0,0
431432,4.0,0.665833,0.712083,0.208954,0.509754,23.294151,71.2083,13.999918,1,0,...,0,1,0,0,0,0,0,1,0,0
739781,14.0,0.470000,0.812917,0.197763,1.778493,14.090000,81.2917,13.250121,1,0,...,0,0,0,0,1,0,0,0,1,0
69785,11.0,0.725000,0.654583,0.154850,0.774344,26.075000,65.4583,10.374950,0,0,...,0,0,0,0,0,0,1,1,0,0


In [6]:
class CustomScaler(TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y):
        self.scaler.fit(X[:, 0:4], y)
        return self
    
    def transform(self, X):
        X_head = self.scaler.transform(X[:, 0:4])
        return np.concatenate((X_head, X[:, 4:]), axis=1)

### Model 1

In [7]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [8]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25, random_state=17)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((827346, 36), (275782, 36), (827346,), (275782,))

In [9]:
scale = CustomScaler()
scale.fit(X1_train, y1_train)
X1_train_scale = scale.transform(X1_train)
X1_test_scale = scale.transform(X1_test)

In [10]:
model1 = LinearRegression()
model1.fit(X1_train_scale, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
pred1 = model1.predict(X1_test_scale)
model1_r = model1.score(X1_test_scale, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test_scale.shape[1]-1)

In [12]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.37098178845335517,
 0.37089966672634034,
 86.346069726282195,
 9.292258591229702)